<a href="https://colab.research.google.com/github/khairiyah98/python_eda/blob/main/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Set working directory
#@title Set up Directory { run: "auto"}
import os
from google.colab import drive 
drive.mount('/content/gdrive')
working_directory = 'My Drive/Shopee' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)

Mounted at /content/gdrive
current directory is : /content/gdrive/My Drive/Shopee


1.	Use pandas function to read the Test_Pandas.xlsx file in: (10 marks)

a.	Assign the result to a variable named “data”

b.	Assign all column names to a variable named “columns”

In [ ]:
import pandas as pd
import xlrd

df = pd.read_excel('Test_Pandas.xlsx')

In [ ]:
df.head()

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34


In [ ]:
df.shape

(464433, 12)

In [ ]:
df.item_creation_date

0        2015-10-01 00:24:42
1        2015-10-01 00:26:37
2        2015-08-25 21:02:13
3        2015-08-30 20:16:07
4        2016-05-09 01:14:34
                 ...        
464428   2017-09-24 18:42:46
464429   2017-09-15 21:10:15
464430   2017-09-17 09:07:06
464431   2017-05-14 11:46:53
464432   2017-06-20 21:48:26
Name: item_creation_date, Length: 464433, dtype: datetime64[ns]

In [ ]:
# list(data) or
list(df.columns)

['itemid',
 'shopid',
 'item_name',
 'item_description',
 'item_variation',
 'price',
 'stock',
 'category',
 'cb_option',
 'is_preferred',
 'sold_count',
 'item_creation_date']

In [ ]:
df.price

0         400.0
1         400.0
2           8.0
3           8.0
4          35.0
          ...  
464428      8.3
464429      0.6
464430      3.0
464431      2.0
464432      4.0
Name: price, Length: 464433, dtype: float64

2.	Use pandas function to find: (20 marks)
a.	How many unique shops are in the dataset?
b.	How many unique preferred and cross border shops are in the dataset?
c.	How many products have zero sold count?
d.	How many products were created in the year 2018?


In [ ]:
#a
df['shopid'].nunique()

7856

In [ ]:
#b
df[(df.cb_option == 1) & (df.is_preferred == 1)]['shopid'].nunique()

158

In [ ]:
#c
df[df.sold_count == 0]['itemid'].nunique()

438092

In [ ]:
#d
df['year']= df['item_creation_date'].dt.year
df[df.year == 2018]['itemid'].nunique()

177407


3.	Use pandas function to find: (10 marks)
a.	Top 3 Preferred shops’ shopid that have the largest number of unique products
b.	Top 3 Categories that have the largest number of unique cross-border products

In [ ]:
#a
df[df.is_preferred == 1].groupby(by="shopid")['itemid'].nunique().sort_values(ascending=False)[0:3]

shopid
43384791    2034
6072241     1998
26974701    1037
Name: itemid, dtype: int64

In [ ]:
#b
df[df.cb_option == 1].groupby(by="category")['itemid'].nunique().sort_values(ascending=False)[0:3]

category
Women's Apparel     80494
Mobile & Gadgets    58760
Men's Wear          51795
Name: itemid, dtype: int64

4.	Find Top 3 shopid with the highest revenue (Assumption: the product price has not been changed.) (15 marks)


In [ ]:
df['revenue'] = df.price * df.sold_count
df.groupby(by="shopid")['revenue'].agg('sum').sort_values(ascending=False)[0:3]

shopid
18796234    177897.79
30769353     63491.39
30628794     39852.00
Name: revenue, dtype: float64


5.	Find number of products that have more than 3 variations (do not include products with 3 or fewer variations) (15 marks)


In [ ]:
import numpy as np
df['variant_count'] = np.where(df['item_variation']== '{}', 0, df.item_variation.str.count(',') + 1)
#df.head()
df[df.variant_count > 3]['itemid'].nunique()
#df[df.variant_count > 3].shape[0]

240976

In [ ]:
df['item_variation'][4]

'{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, NIKE SWOOSH BLACK: 35.0, NIKE SMALL SWOOSH: 35.0, NEWERA WHITE: 35.0, NEWERA MAROON: 35.0}'


6.	Use pandas function to identify duplicated listings within each shop (If listing A and B in shop S have the exactly same product title, product detailed description, and price, both listing A and B are considered as duplicated listings) (30 marks)

a.	Mark those duplicated listings with True otherwise False and store the marking result in a new column named “is_duplicated” 

b.	Find duplicate listings that has less than 2 sold count and store the result in a new excel file named “duplicated_listings.xlsx”

c.	Find the preferred shop shopid that have the most number of duplicated listings

In [ ]:
#a
# Find a duplicate rows
dup_col = df.duplicated(subset=['item_name', 'item_description', 'price'])
df['is_duplicated'] = dup_col
df.head()
#df[df.duplicated(['item_name', 'item_description', 'price'])] #to double check

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date,year,is_duplicated
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42,2015,False
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37,2015,False
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13,2015,False
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07,2015,False
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34,2016,False


In [ ]:
#b
duplicated_listings = df[(df.is_duplicated == True) & (df.sold_count < 2)]
!pip install xlsxwriter
df.to_excel('duplicated_listings.xlsx', engine='xlsxwriter')

In [ ]:
#c - find the preferred shop shopid that have the most number of duplicated listings
df[(df.is_preferred == 1) & (df.is_duplicated == True)].groupby(by="shopid").size().sort_values(ascending=False)[0:3]

shopid
43384791    1933
6072241      791
8090602      434
dtype: int64